In [ ]:
import time

import matplotlib as plt
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
# import seaborn as sns

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import sklearn
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score
from sklearn.model_selection import train_test_split

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def data_collect():
    s = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    #driver.get('https://www.google.com')
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    # , options=chrome_options
    
    # driver = webdriver.Chrome("C:/Users/yuval/Desktop/Shahar/Data-science-Weather/chromedriver/chromedriver", options=chrome_options)
    # C:/Users/yuval/Desktop/Shahar/Data-science-Weather/chromedriver/chromedriver --> Shahar
    # /Users/gal.bachar/git_wa/HIT/Data-science-Weather/chromedriver/chromedriver  --> Gal
    df = pd.DataFrame([], columns=["year", "month", "day", "temp","temp min","temp max","dew point", "humidity", "windspeed", "windspeed min", "windspeed max", "precipitation"])
    df_row_count=1

    # switch temp from fahrenheit to celsius
    driver.get(f"https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/1950-1")
    time.sleep(2)
    settings = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "wuSettings")))
    settings.click()
    time.sleep(2)
    celsius = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH,'//*[@id="wuSettings-quick"]/div/a[2]')))
    celsius.click()

    for year in range(1950,2023):
        for month in range(1,13):
            try:
                driver.get(f"https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/{year}-{month}")
                print(f'{month}-{year}')
            except Exception as e:
                print(f'{e} - unable to get: {month}.{year}')
            try:
                table_id= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "days")))
                data = table_id.find_elements(By.TAG_NAME, "table") # get all of the rows in the table
                num_of_days = len(data[1].find_elements(By.TAG_NAME, "tr"))

                for i in range(1,num_of_days):
                    day = int(data[0].find_elements(By.TAG_NAME, "tr")[i].text)
                    temp = float(data[1].find_elements(By.TAG_NAME, "tr")[i].text.split(' ')[1])
                    temp_min = float(data[1].find_elements(By.TAG_NAME, "tr")[i].text.split(' ')[2])
                    temp_max = float(data[1].find_elements(By.TAG_NAME, "tr")[i].text.split(' ')[0])
                    dew_point = float(data[2].find_elements(By.TAG_NAME, "tr")[i].text.split(' ')[1])
                    humidity = float(data[3].find_elements(By.TAG_NAME, "tr")[i].text.split(' ')[1])
                    windspeed = float(data[4].find_elements(By.TAG_NAME, "tr")[i].text.split(' ')[1])
                    windspeed_min = float(data[4].find_elements(By.TAG_NAME, "tr")[i].text.split(' ')[2])
                    windspeed_max = float(data[4].find_elements(By.TAG_NAME, "tr")[i].text.split(' ')[0])
                    precipitation = float(data[6].find_elements(By.TAG_NAME, "tr")[i].text)
                    day_data = [year,month,day,temp,temp_min,temp_max,dew_point,humidity,windspeed,windspeed_min,windspeed_max,precipitation]
                    df.loc[df_row_count] = day_data
                    df_row_count += 1
            except Exception as e:
                print(f"cloud not get {year}-{month} data: {e}")

    driver.quit()
    df.to_csv('~/Documents/nyc_weather.csv',index=False) #for debugging
    return df

In [ ]:
def normalize_data(data_df, good_weather_values):

    # Normalize each column to a float vlaue between 0 - 1 according its "Good_weather_values" definition. 
    # When 0 means "bad" weather and 1 means "Good" weather 
    

    norm_df = data_df.copy()
    # Calculating only relevant values for our target column
    for i in norm_df.columns:
        if i not in ['year', 'month', 'day',"temp min","temp max", "windspeed min", "windspeed max","dew point"]:
            good_min = good_weather_values[i][0]
            good_max = good_weather_values[i][1]
            diff_min = good_weather_values[i][0] - norm_df[i].min()
            diff_max = norm_df[i].max() - good_weather_values[i][1]
            norm_df.loc[(norm_df[i] < good_min), i] = 1 - ((good_min - norm_df[i]) / diff_min)
            norm_df.loc[(norm_df[i] > good_max), i] = 1 - ((norm_df[i] - good_max) / diff_max)
            norm_df.loc[(norm_df[i] <= good_max) & (norm_df[i] >= good_min), i] = 1
    
    # Creating a Target column
    norm_df.insert(7, "pleasant day", 0)
    
    # Calulating target column for each day base on average of temp, humidity, windspeed and precipitation values 
    norm_df.loc[(norm_df["temp"]!=0),"pleasant day"] = (norm_df["temp"] + norm_df["humidity"] + norm_df["windspeed"] + norm_df["precipitation"]) / 4

    # In case there is a NAN value in one of the relevant columns the average is done base on the other 3 columns
    norm_df.loc[(norm_df["precipitation"].isna()), "pleasant day"] = (norm_df["temp"] + norm_df["humidity"] + norm_df["windspeed"]) / 3
    norm_df.loc[(norm_df["temp"].isna()), "pleasant day"] = (norm_df["precipitation"] + norm_df["humidity"] + norm_df["windspeed"]) / 3
    norm_df.loc[(norm_df["humidity"].isna()), "pleasant day"] = (norm_df["temp"] + norm_df["precipitation"] + norm_df["windspeed"]) / 3
    norm_df.loc[(norm_df["windspeed"].isna()), "pleasant day"] = (norm_df["temp"] + norm_df["humidity"] + norm_df["precipitation"]) / 3
    
    # norm_df.drop(columns=["precipitation"],inplace=True)
    # norm_df.dropna(inplace=True)
    
    return norm_df

In [ ]:
def remove_missing_data(data):
    data.loc[data["year"] <= 2014, "precipitation"] = np.NaN
    return data

In [ ]:
def num_of_good_days_per_month(data):
    fig, axes = plt.subplots(2, 1, figsize=(20, 10))
    months=['Jan', 'Feb','Mar','Apr','May','jun','Jul','Aug','Sep','Oct','Nov','Dec']
    res = {}
    for i in range(0,12):
        res[months[i]] = len(data.loc[(data['month'] == (i+1)) & (data['pleasant day'] >= 0.75)])
    ser = pd.Series(res)
    # print(ser)
    ser.plot(kind='pie', ax=axes[0], title = "All time good days per month")
    ser.plot(kind='line', ax=axes[1], title = "Pleasent days per month")

In [ ]:
def num_of_good_days_trends(data):
    fig, axes = plt.subplots(2, 1, figsize=(20, 10))
    months=['Jan', 'Feb','Mar','Apr','May','jun','Jul','Aug','Sep','Oct','Nov','Dec']
    res = {}
    for j in range(1950,2022):
        for i in range(0,12):
            #print(data.loc[((data['month'] == (i+1)) & (data['year'] == j)),'pleasant day'].mean())
            res[f"{i + 1}-{j}"] = round(data.loc[((data['month'] == (i+1)) & (data['year'] == j)),'pleasant day'].mean(),2)
    ser = pd.Series(res)
    ser.plot(kind='line', ax=axes[0], title = "Mean good days per month")

    ser2 = pd.Series(data["pleasant day"])
    ser2.plot(kind='line', ax=axes[1], title = "Pleasant days values over the time")
    ser2.loc[ser2>=0.75].plot(kind='line', ax=axes[1],linestyle="-")

In [ ]:
def get_highly_correlated_cols(data):
    col_correlated = set()
    tuple_array = []
    correlations = []
    for i in range (len(data.corr().columns)):
        for j in range(i):
            if(data.corr().iloc[j, i] >= 0.5) and (data.corr().columns[i] not in col_correlated):
                correlations.append(data.corr().iloc[i, j])
                tuple_array.append([j, i])
    return correlations, tuple_array

In [ ]:
def high_correlated_scatters(data):
    fig, axes = plt.subplots(len(correlations), 1, figsize = (10, 12),constrained_layout=True)
    axe_i = 0
    index = np.argsort(correlations)
    columns = list(data.columns)
    for n_correlation in index:
        col_lt, col_rt = tuple_arr[n_correlation]
        col_left_title, col_right_title = columns[col_lt], columns[col_rt]
        title = "corr('%s', '%s') = %4.2f" % (col_left_title, col_right_title, correlations[n_correlation])
        data.plot(x = col_left_title, y = col_right_title, kind = "scatter",
                ax = axes[axe_i], title = title, xlabel = col_left_title, ylabel = col_right_title)
        axe_i = axe_i + 1

In [ ]:
def remove_corrupt_rows(df, num_max_missing_cols):
    cp = df.dropna(thresh=len(df.columns)-num_max_missing_cols).copy()
    return cp

In [ ]:
def getOutliers(df):
    #Working on a copy DF
    cp = df.copy()
    all_outliers = {}
    #Plots for each column and its outliers
    fig, axes = plt.subplots(1, len(cp.columns) - 3, figsize = (30, 5))
    axe_i = 0
    #Itterating over all columns 
    for col in cp.columns:
        # After few tests, we found out that the best Z_score_value for humidity is 2.5 
        if col in ["humidity"]:
            z_score_value=2.5
        else:
            #Other z_score_values should be 3 
            z_score_value=3
        #Ignorring Year, Month and Day columns
        if col not in ["year", "month", "day"]:    
            #Creating Histogram
            ax = cp[col].plot(bins=50, ax=axes[axe_i], kind="hist")
            axe_i = axe_i + 1
            ax.set(xlabel=col)
            plt.ylabel("Frequency")
            # looking for outliers using distances of standard deviation from the mean.
            z_score = (cp[col] - cp[col].mean()) / cp[col].std()
            
            outliers = abs(z_score) > z_score_value # after some tests if we will search for lower distance we will lost important information.
            print (f"Number of outliers for {col} - {sum(outliers)}")
            all_outliers[col] = outliers

    #return a dictinary with all columns and outliers
    return all_outliers


In [ ]:
def putNanOutliers(df, all_outliers):
    # Replacing outliers with NAN
    cp = df.copy()
    for key in all_outliers:
        cp.loc[all_outliers[key].tolist(),key] = np.NaN
    return cp

In [ ]:
def showOutliers(all_outliers):
    fig, axes = plt.subplots(1, len(df.columns) - 3, figsize = (30, 5))
    axe_i = 0
    for col in all_outliers:
        ser = pd.Series(df.loc[all_outliers[col],col])
        ser.plot(kind="hist", ax=axes[axe_i], title = f"Outliers for {col}")
        axe_i = axe_i + 1

In [ ]:
def getArray(data):
    arr = []
    for i in range(0,len(data)):
        arr.append(data[i])
    return arr

In [ ]:
def graphOverTime(df,param):
    colors=["r","b","g","black"]
    i=0
    for col in param:
        c=colors[i]
        i=i+1
        df.plot(kind="line",x="year", y=[col],figsize=(15, 7), grid=True,color=c)

In [ ]:
# df = data_collect()
df = pd.read_csv("~/Documents/nyc_weather.csv")
print ("Done")

In [ ]:
print("All data")
df

In [ ]:
print("Dataframe describe")
df.describe(include="all")

In [ ]:
all_outliers = getOutliers(df)
fixed_data = remove_missing_data(df)
noOut = putNanOutliers(fixed_data,all_outliers)

df_no_outliers = remove_corrupt_rows(noOut, 1)
print("Full DataFrame")
print(df.shape)
print("No outliers DataFrame")
print(df_no_outliers.shape)

In [ ]:
# Show outliers
showOutliers(all_outliers)

In [ ]:
df_no_outliers = remove_corrupt_rows(noOut, 1)
print("Full DataFrame")
print(df.shape)
print("No outliers DataFrame")
print(df_no_outliers.shape)

In [ ]:
param = ["temp","humidity","windspeed"]
graphOverTime(df_no_outliers,param)

In [ ]:
# nice weather:
# temp 17-20
# humidity 30-40%
# windspeed 4-7
# precipitation 0-1
good_weather_values = {'temp': [17,20], 'humidity':[30,40],'windspeed':[4,7], 'precipitation':[0,1]}

norm_df = normalize_data(df_no_outliers, good_weather_values)
print("All data - normalized")
norm_df

In [ ]:
num_of_good_days_per_month(norm_df)

In [ ]:
num_of_good_days_trends(norm_df)

In [ ]:
corr_df = norm_df.drop(columns=['temp min','temp max','windspeed min','windspeed max'])

correlations, tuple_arr = get_highly_correlated_cols(corr_df)
high_correlated_scatters(corr_df)

In [ ]:
def get_df_target_col(df):
    y= df['pleasant day']
    cp_df = df.drop('pleasant day',axis=1)
    return cp_df,y

def train_1st_model(X_train, y_train):
    reg=LinearRegression()
    
    return reg.fit(X_train,y_train)
def predict_1st(trained_1st_model, X_test):
    predict=trained_1st_model.predict(X_test)
    return predict

In [ ]:
train, test = train_test_split(norm_df.loc[(norm_df["month"]==5) & (norm_df["day"]==10)], test_size=0.25)

In [ ]:
X_1st_train, y_1st_train = get_df_target_col(train)
X_1st_test, y_1st_test = get_df_target_col(test)
# print(X_1st_train.isna().sum())

trained_model_1st = train_1st_model(X_1st_train, y_1st_train)
pred_1st_vals = predict_1st(trained_model_1st, X_1st_test)
print(pred_1st_vals.mean(),y_1st_test.mean())

In [ ]:
print(pred_1st_vals,y_1st_test)

In [ ]:
print(y_1st_test)